Import the neccessary libraries

In [2]:
import numpy as np
import pandas as pd
import os
from imageio import imread
from skimage.transform import resize
import datetime
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, TimeDistributed, GRU, LSTM, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.backend import clear_session
import random as rn
import warnings
warnings.filterwarnings("ignore")

Set the random seed so that the results don't vary drastically each time the code is run

In [3]:
np.random.seed(100)
rn.seed(100)
tf.random.set_seed(100)

Download and unzip the dataset

In [5]:
!gdown https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
!unzip ./Project_data.zip

In [6]:
!rm ./Project_data.zip

Folder names for the training and validation data are read into train_doc and val_doc respectively. Set the batch size such that GPU memory is utilized to its full capacity

In [7]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 50

In [8]:
class_label = [folder.split(";")[2].strip("\n") for folder in train_doc]
pd.Series(class_label).value_counts()

There is no class imbalance as the no of samples is almost the same for each class. Hence accuracy can used as a metric to evaluate the performance of the model

## Generator

*   Build a data generator that generates batches of videos of size = batch_size
*   Since all images don't have the same shape, they need to be resized.
*   Not all frames of each video need to be included in the training data
*   The images need to be normalised before they are fed to the model



 

In [9]:
img_idx = np.arange(0, 30, 2) # list of indices of the frames that you want to use from each video
def generator(source_path, folder_list, batch_size, ablation = None):
    print( 'Source path = ', source_path, 'batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list) # shuffle the data for each epoch
        if ablation is None :
          num_batches = len(t) // batch_size # calculate the number of batches
          for batch in range(num_batches): # iterate over the number of batches
            batch_data = np.zeros((batch_size, len(img_idx), 120, 120, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size, 5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image = resize(image, (120, 120, 3)) # resize each image

                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0])) #centering the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1])) #centering the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2])) #centering the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels 
          
          # for the remaining sequences
          if len(t) % batch_size != 0 : 
            rem_data = len(t) % batch_size
            batch_data = np.zeros((rem_data, len(img_idx), 120, 120, 3))
            batch_labels = np.zeros((rem_data, 5))
            for folder in range(rem_data) :
              imgs = os.listdir(source_path + '/' + t[folder + (num_batches * batch_size)].strip().split(";")[0])
              for idx, item in enumerate(img_idx) :
                image = imread(source_path+'/'+ t[folder + (num_batches * batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (120, 120, 3))
                batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0])) #centering the image
                batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1])) #centering the image
                batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2])) #centering the image
              batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels     

            
        else :
          num_batches = ablation // batch_size
          for batch in range(num_batches) :
            batch_data = np.zeros((batch_size, len(img_idx), 120, 120, 3))
            batch_labels = np.zeros((batch_size, 5))
            for folder in range(batch_size) :
              imgs = os.listdir(source_path + '/' + t[folder + (batch*batch_size)].strip().split(";")[0])
              for idx, item in enumerate(img_idx) :
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (120, 120, 3))
                batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0])) #centering the image
                batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1])) #centering the image
                batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2])) #centering the image
              batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
          yield batch_data, batch_labels
        
          if ablation % batch_size != 0 :
            rem_data = ablation % batch_size
            batch_data = np.zeros((rem_data, len(img_idx), 120, 120, 3))
            batch_labels = np.zeros((rem_data, 5))
            for folder in range(rem_data) :
              imgs = os.listdir(source_path + '/' + t[folder + (num_batches * batch_size)].strip().split(";")[0])
              for idx, item in enumerate(img_idx) :
                image = imread(source_path+'/'+ t[folder + (num_batches * batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image = resize(image, (120, 120, 3))
                batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.mean(image[:,:,0])) #centering the image
                batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.mean(image[:,:,1])) #centering the image
                batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.mean(image[:,:,2])) #centering the image
              batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels     
        


In [10]:
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

Looking at a random frame from a random video generated by the generator

In [11]:
batch_data, batch_labels = next(generator(train_path, train_doc, batch_size))
plt.figure(figsize = (5, 5))
vid_rand = np.random.randint(batch_size)
frame_rand = np.random.randint(len(img_idx))
plt.imshow(batch_data[vid_rand, frame_rand, :, :, :])
plt.axis("off")
plt.show()

## Modelling
### Design the model in such a way that its accuracy is high with least number of parameters, so that it can fit in the memory of the webcam and respond instantly in real time scenarios

### Defining a function that trains a model

It does the following :

*   Defines the optimiser
*   Compiles the model
*   Defines callbacks such as ModelCheckpoint for saving the best model at the end of every epoch and ReduceLROnPlateau reducing the learning rate once the val_loss plateaus
*   Creates the generator objects for the training data and validation data
*   Calls the fit() method of the keras model
*   Plots the validation accuracy and training accuracy across epochs

In [12]:
def model_training(model, lr, patience, factor, num_epochs = 30) :
  adam = optimizers.Adam(learning_rate = lr)#write your optimizer
  model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

  train_generator = generator(train_path, train_doc, batch_size)
  val_generator = generator(val_path, val_doc, batch_size)
  
  curr_dt_time = datetime.datetime.now()
  model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
  if not os.path.exists(model_name):
    os.mkdir(model_name)
        
  filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

  checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto',save_freq = 'epoch')
  LR = ReduceLROnPlateau(patience = patience, factor = factor, verbose = 1) 
  callbacks_list = [checkpoint, LR]

  if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
  else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

  if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
  else:
    validation_steps = (num_val_sequences//batch_size) + 1

  history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=[callbacks_list], validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
  
  plt.figure(figsize = (8, 8))
  plt.plot(range(num_epochs), history.history["categorical_accuracy"])
  plt.plot(range(num_epochs), history.history["val_categorical_accuracy"])
  plt.legend(labels = ["train", "validation"])
  plt.show()

### model_1 : CONV3D Model

In [13]:
clear_session()
#write your model here
model_1 = Sequential()
model_1.add(Conv3D(32, 3, activation = "elu", input_shape = (len(img_idx), 120, 120, 3), padding = "same"))
model_1.add(MaxPooling3D(padding = "same"))
model_1.add(BatchNormalization())
model_1.add(Conv3D(64, 3, activation = "elu", padding = "same"))
model_1.add(MaxPooling3D(padding = "same"))
model_1.add(BatchNormalization())
model_1.add(Conv3D(128, 3, activation = "elu"))
model_1.add(MaxPooling3D())
model_1.add(BatchNormalization())
model_1.add(Flatten())
model_1.add(Dense(256, activation = "elu"))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.7))
model_1.add(Dense(512, activation = "elu"))
model_1.add(BatchNormalization())
model_1.add(Dropout(0.7))
model_1.add(Dense(5, activation = "softmax"))
model_1.summary()

### Training model_1

In [14]:
model_training(model_1, lr = 0.001, patience = 1, factor = 0.8)

### model_2 : CNN_GRU Model

In [15]:
clear_session()

resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = (120, 120, 3))
output = resnet.layers[-1].output
output = Flatten()(output)

resnet = Model(resnet.input, output)

for layer in resnet.layers :
  layer.trainable = False

model_2 = Sequential()
model_2.add(Input(shape = (len(img_idx), 120, 120, 3)))
model_2.add(TimeDistributed(resnet))
model_2.add(TimeDistributed(BatchNormalization()))
model_2.add(GRU(64, return_sequences = True))
model_2.add(TimeDistributed(BatchNormalization()))
model_2.add(GRU(32))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.5))
model_2.add(Dense(16, activation = "elu"))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.5))
model_2.add(Dense(5, activation = "softmax"))
model_2.summary()

### Training model_2

In [16]:
model_training(model_2, lr = 0.001, patience = 1, factor = 0.8)



*   model_1 and model_2 have similar validation accuracies
*   model_2 has around 5 times the no of parameters as model_1

Since the model has to be deployed in a real world scenario, it is better to go with model_1(CONV3D model) which has a considerably lower no of parameters